In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import pandas as pd
import re
import time
from tqdm.notebook import tqdm

In [2]:
url = 'https://in.tgstat.com/ru/tag/investments?ysclid=ltxecd5p1e666733084'
res = requests.get(url, headers={'User-Agent': UserAgent().chrome})

In [3]:
soup = BeautifulSoup(res.content)

In [4]:
channel_blocks = soup.find_all('div', {'class':r'col-12 col-sm-6 col-lg-4'})

In [5]:
names = []
subscribers = []
links = []

for block in channel_blocks:
    names.append(block.find('div', {'class' : r'font-16 text-dark text-truncate'}).text)
    subscribers.append(int(block.find('div', {'class' : r'font-12 text-truncate'})\
                      .text.strip()[:-12].replace(' ', '')))
    link = block.find(attrs={'class' : r'text-body'}, href=True)['href']
    
    dog_sigh_pos = link.find('@')
    slash_pos = [m.start() for m in re.finditer('/', link)][-1]
    
    if dog_sigh_pos != -1:
        link = 'https://t.me/' + link[dog_sigh_pos+1:]
    else:
        link = 'https://t.me/' + link[slash_pos+1:]
        
    links.append(link)

In [6]:
channels_info = pd.DataFrame({'name' : names,
                             'subscribers' : subscribers, 
                             'link' : links})

channels_info.head(10)

,name,subscribers,link
0,Тинькофф Инвестиции,669794,https://t.me/tinkoff_invest_official
1,Альфа-Инвестиции,421969,https://t.me/alfa_investments
2,Мои Инвестиции,288064,https://t.me/omyinvestments
3,bitkogan,285107,https://t.me/bitkogan
4,СберИнвестиции,232507,https://t.me/SberInvestments
5,Start The History,224910,https://t.me/Hj9bW7gkrulhMGMy
6,InvestFuture,194692,https://t.me/investfuture
7,Кубышка | Евгений Марченко,174189,https://t.me/AAAAAELQAc1vKST8oZwVxg
8,Лимон на чай,153156,https://t.me/lemonfortea
9,РынкиДеньгиВласть | РДВ,149994,https://t.me/AK47pfl


In [9]:
from telethon import TelegramClient
from telegram import config

In [10]:
api_id = config.API_ID
api_hash = config.API_HASH
name = 'test'

In [11]:
client = TelegramClient(name, api_id, api_hash)
await client.start()

In [12]:
from telethon.tl.functions.messages import GetHistoryRequest

In [52]:
cnt = 0
msg = []
channels_info['is_restricted'] = True
channels_info['error_while_parsing'] = False


for i, channel in enumerate(tqdm(channels_info.to_dict('records'))):
    try:
        url = channel['link']
        tg_channel = await client.get_entity(url)

        history = await client(GetHistoryRequest(
            peer=tg_channel,
            offset_id=0,
            offset_date=None, add_offset=0,
            limit=1, max_id=0, min_id=0,
            hash=0))

        if history.messages[0].to_dict()['message'] != '':
            channels_info.at[i, 'is_restricted'] = False
    except:
        channels_info.at[i, 'error_while_parsing'] = True

  0%|          | 0/10 [00:00<?, ?it/s]

In [56]:
channels_info = channels_info[~channels_info['is_restricted'] & ~channels_info['error_while_parsing']]\
                            [['name', 'subscribers', 'link']]\
                            .reset_index(drop=True)
channels_info.head(10)

,name,subscribers,link
0,Альфа-Инвестиции,421969,https://t.me/alfa_investments
1,Мои Инвестиции,288064,https://t.me/omyinvestments
2,bitkogan,285107,https://t.me/bitkogan
3,СберИнвестиции,232507,https://t.me/SberInvestments
4,Лимон на чай,153156,https://t.me/lemonfortea
5,РынкиДеньгиВласть | РДВ,149994,https://t.me/AK47pfl


In [58]:
channels_info.to_csv('../../data/telegram/channels.csv')

In [24]:
# me = await client.get_dialogs()

# async def dump_all_messages(channel):
# 	"""Записывает json-файл с информацией о всех сообщениях канала/чата"""
# 	offset_msg = 0    # номер записи, с которой начинается считывание
# 	limit_msg = 100   # максимальное число записей, передаваемых за один раз

# 	all_messages = []   # список всех сообщений
# 	total_messages = 0
# 	total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

# 	class DateTimeEncoder(json.JSONEncoder):
# 		'''Класс для сериализации записи дат в JSON'''
# 		def default(self, o):
# 			if isinstance(o, datetime):
# 				return o.isoformat()
# 			if isinstance(o, bytes):
# 				return list(o)
# 			return json.JSONEncoder.default(self, o)

# 	while True:
# 		history = await client(GetHistoryRequest(
# 			peer=channel,
# 			offset_id=offset_msg,
# 			offset_date=None, add_offset=0,
# 			limit=limit_msg, max_id=0, min_id=0,
# 			hash=0))
# 		if not history.messages:
# 			break
# 		messages = history.messages
# 		for message in messages:
# 			all_messages.append(message.to_dict())
# 		offset_msg = messages[len(messages) - 1].id
# 		total_messages = len(all_messages)
# 		if total_count_limit != 0 and total_messages >= total_count_limit:
# 			break

# 	with open('channel_messages.json', 'w', encoding='utf8') as outfile:
# 		 json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)


# async def main():
# 	url = input("Введите ссылку на канал или чат: ")
# 	channel = await client.get_entity(url)
# 	await dump_all_participants(channel)
# 	await dump_all_messages(channel)


# with client:
# 	client.loop.run_until_complete(main())